Изучите признаки, проведите правильный описательный анализ признаков.

Удалите / заполните / замените пропуски или ошибки в данных.

Обязательно проверьте наличие дубликатов В СТРОКАХ и удалите, если они есть (команды в pandas: duplicated, drop_duplicates). Дублем является тот объект, который является идентичный другому (то есть одна строка равна другой).
Выполните следующую аналитику:

1. Какие самые высокооплачиваемые должности? Есть ли выбросы в заработной плате? В каких должностях?
2. Влияет ли удаленная работа на зарплату? Есть ли корреляция?
3. Зависит ли заработная плата от размера предприятия?
4. Какие страны наиболее популярны среди вакансий? В каких заработная плата выше, а в каких ниже?
5. Оцените медианную зарплату для каждой должности.
6. Оцените, как сильно изменились зарплаты в 2023 году, относительно 2020. Какие должности стали более оплачиваемы, а какие нет?
7. Выявите портрет самого успешного и богатого работника: кем он должен работать, где проживать, удаленная ли работа.
8. Много ли людей работают и проживают в разных регионах? Где больше всего людей проживает и много ли из них (в процентном соотношении) работают удаленно на другую страну?
9. Выявите новые инсайты, если захотите! Творите!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik
from phik import phik_matrix

pd.options.display.max_columns = None # полностью отображаем столбцы


In [ ]:
df = pd.read_csv('ds_salaries.csv')

df.info()

In [ ]:
df.sample(100) 

In [ ]:
df.describe().T

In [ ]:
df.describe(include='object').T

In [ ]:
df.isnull().sum()
#Смотрим сумму пропусков по всем колонкам. Пропусков не обнаружено

In [ ]:
df.duplicated().sum()

1171 дубликатов строк

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

### Какие самые высокооплачиваемые должности?

In [ ]:
mean_salaries = df.groupby(['job_title'])['salary_in_usd'].mean().sort_values(ascending=False)
plt.figure(figsize=(10,20))
ax = sns.barplot(x=mean_salaries, y=mean_salaries.index, orient='h', order=mean_salaries.index)
plt.title('Average Salary by Job Title (Sorted by Mean Salary)')
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
display(f'Самые высокооплачиваемые должности - {list(mean_salaries.index[:5])}')

### Есть ли выбросы в заработной плате? В каких должностях?

In [ ]:
sorted_job_titles = mean_salaries.index

plt.figure(figsize=(10,20))
ax = sns.boxplot(data = df, x = 'salary_in_usd', y = 'job_title', order=sorted_job_titles, width=.5)
plt.title('Box Plot of Salaries by Job Title (Sorted by Mean Salary)')
plt.show()

In [ ]:
outliers = pd.DataFrame(columns=['job_title', 'salary_in_usd'])

for job in df['job_title'].unique():
    job_salaries = df[df['job_title'] == job]['salary_in_usd']
    Q1 = job_salaries.quantile(0.25)
    Q3 = job_salaries.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    job_outliers = job_salaries[(job_salaries < lower_bound) | (job_salaries > upper_bound)]
    outliers = pd.concat([outliers, pd.DataFrame({'job_title': job, 'salary_in_usd': job_outliers})])


In [ ]:
print("Должности с выбросами:")
print(outliers['job_title'].unique())
print("\nТаблица")
print(outliers)


### Влияет ли удаленная работа на зарплату? Есть ли корреляция?

In [ ]:
corr_matrix = df[['salary_in_usd', 'remote_ratio']].phik_matrix()

# Печатаем корреляционную матрицу
print("Корреляционная матрица:")
print(corr_matrix)

In [ ]:
df.head()

In [ ]:

sns.heatmap(df[['salary_in_usd', 'remote_ratio']].phik_matrix (interval_cols=['salary_in_usd'])[['remote_ratio']], annot=True)
plt.show()

Удалённая работа влияет на зарплату. Очень слабая корреляция.

In [ ]:
plt.figure(figsize=(10, 5))

sns.lineplot(x=df.work_year, y=df.salary_in_usd, hue=df.remote_ratio)
plt.show()

### Зависит ли заработная плата от размера предприятия?

In [ ]:
sns.heatmap(df[['salary_in_usd', 'company_size']].phik_matrix (interval_cols=['salary_in_usd'])[['company_size']], annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

sns.lineplot(x=df.work_year, y=df.salary_in_usd, hue=df.company_size)
plt.show()

В среднем, зарплата в большой компании выше чем в маленькой. 

### Какие страны наиболее популярны среди вакансий? В каких заработная плата выше, а в каких ниже?

In [ ]:
temp = df.groupby(['company_location'])['salary_in_usd'].agg('mean').sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(5,15))
ax = sns.barplot(x=temp, y=temp.index, orient='h', order=temp.index)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
display(f'Пять стран с самой высокой средней зарплатой - {list(temp.index[:5])}')

In [ ]:
plt.figure(figsize=(5,15))
ax = sns.countplot(data = df, y='company_location', orient='v', order=df['company_location'].value_counts().index)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
print(f'Самые популярные страны среди вакансий : {list(df['company_location'].value_counts()[:5].index)}')

### Оцените медианную зарплату для каждой должности.

In [ ]:
sorted_by_median = df.groupby(['job_title'])['salary_in_usd'].median().sort_values(ascending=False)

plt.figure(figsize=(10,20))
ax = sns.boxplot(data = df, x = 'salary_in_usd', y = 'job_title', order = sorted_by_median.index, width=.5)
plt.title('Box Plots of Salaries with medians by Job Title (Sorted by median salary)')
plt.show()

In [ ]:
df_sorted_by_median = sorted_by_median.reset_index()
df_sorted_by_median.rename(columns={'salary_in_usd': 'median_salary'}, inplace=True)
df_sorted_by_median.head(5)

### Оцените, как сильно изменились зарплаты в 2023 году, относительно 2020. Какие должности стали более оплачиваемы, а какие нет?

# Под вопросом, не понимаю как такое количество вакансий сделать в более адекватном формате сравнения

In [ ]:
temp = df.groupby(['job_title', 'work_year'])['salary_in_usd'].agg('mean').reset_index()

plt.figure(figsize=(30,30))
sns.lineplot(x=temp['work_year'], y=temp['salary_in_usd'], hue=temp['job_title'])
plt.show()

In [ ]:
plt.figure(figsize=(20,30))
sns.boxplot(data = df, x = 'salary_in_usd', y = 'job_title', hue = 'work_year', width = 2, palette ='bright')
plt.show()

### Выявите портрет самого успешного и богатого работника: кем он должен работать, где проживать, удаленная ли работа.

По прошлым представлениям графиков уже известно, что это должен быть работник на позиции 'Data Science Tech Lead', 'Cloud Data Architect', 'Data Lead', 'Data Analytics Lead', 'Principal Data Scientist', проживать в 'US', 'GB', 'CA', 'IN', 'DE' и желательно не удалённо, но не обязательно.

### Много ли людей работают и проживают в разных регионах? Где больше всего людей проживает и много ли из них (в процентном соотношении) работают удаленно на другую страну?

In [ ]:
df['ratio_location_different'] = (df['employee_residence'] != df['company_location']) & (df['remote_ratio'] != 0)
plt.figure(figsize=(20,5))
sns.countplot(data = df, x = 'employee_residence', hue = 'ratio_location_different')
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(data = df, x = 'company_location', hue = 'ratio_location_different')
plt.show()

In [ ]:
overall_ratio_location_different = (df['ratio_location_different'] == True).agg("sum")

In [ ]:
print(f'{overall_ratio_location_different} людей работают и проживают в разных регионах. Это {round((overall_ratio_location_different / df.shape[0]) * 100, 2)}% от общего количества.')

In [ ]:
US_ratio_location_different = df[(df['employee_residence'] == 'US') & (df['ratio_location_different'] == True)].shape[0]

In [ ]:
print(US_ratio_location_different)

In [ ]:
print(f'Больше всего проживают в US. Работают удаленно на другую страну {US_ratio_location_different} человек, это {round(US_ratio_location_different / (df['employee_residence'] == 'US').shape[0] * 100, 2)}% от общего количества.')